# Level 3: Advanced Agent Capabilities with Prompt Chaining and ReAct Agent

Building on the simple agent introduced in [Level 2](Level2_simple_agent_with_websearch.ipynb), this tutorial continues the agent-focused section of our series by introducing techniques that make the agent smarter and more autonomous: **Prompt Chaining** and the **ReAct (Reasoning + Acting) framework**. These approaches allow the agent to complete multi-step tasks, dynamically choose tools, and adjust its behavior based on context.

- **Prompt Chaining** connects multiple prompts into a coherent sequence, allowing the agent to maintain context and perform multi-step reasoning across tool invocations. 
- **ReAct Agent** combines reasoning and acting steps in a loop, enabling the agent to make decisions, use tools dynamically, and adapt based on intermediate results. 

## Overview

In this notebook, you'll explore three agent configurations:
1. **Simple Agent (Baseline)** – Uses a single web search tool.
2. **Prompt Chaining** – Performs structured, multi-step reasoning by chaining prompts and responses.
3. **ReAct Agent** – Dynamically plans and executes actions using a loop of reasoning and tool use.


## Prerequisites

Before starting this notebook, ensure that you have:
- Followed the instructions in the [Setup Guide](./Level0_getting_started_with_Llama_Stack.ipynb) notebook. 
- A Tavily API key. It is critical for this notebook to run correctly. You can register for one at [https://tavily.com/](https://tavily.com/).

## 1. Setting Up this Notebook
We will start with a few imports needed for this demo only.

In [2]:
! pip install geocoder


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from llama_stack_client import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
import sys
sys.path.append('..') 
from src.client_tools import get_location

Next, we will initialize our environment as described in detail in our ["Getting Started" notebook](./Level0_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [4]:
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# pretty print of the results returned from the model/agent
import sys
sys.path.append('..')  
from src.utils import step_printer
from termcolor import cprint

base_url = os.getenv("REMOTE_BASE_URL")


# Tavily search API key is required for some of our demos and must be provided to the client upon initialization.
# We will cover it in the agentic demos that use the respective tool. Please ignore this parameter for all other demos.
tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY")
if tavily_search_api_key is None:
    provider_data = None
else:
    provider_data = {"tavily_search_api_key": tavily_search_api_key}


client = LlamaStackClient(
    base_url=base_url,
    provider_data=provider_data
)
    
print(f"Connected to Llama Stack server")

# model_id for the model you wish to use that is configured with the Llama Stack server
model_id = "granite32-8b"

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "False")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

Connected to Llama Stack server
Inference Parameters:
	Model: granite32-8b
	Sampling Parameters: {'strategy': {'type': 'greedy'}, 'max_tokens': 512}
	stream: False


## 2. Simple Agent (Baseline)
Same agent setup as [Level 2 notebook](Level2_simple_agent_with_websearch.ipynb). 

In [5]:
agent = Agent(
    client, 
    model=model_id,
    instructions="""You are a helpful websearch assistant. When you are asked to search the latest you must use a tool. 
            Whenever a tool is called, be sure return the response in a friendly and helpful tone.
            """ ,
    tools=["builtin::websearch"],
    sampling_params=sampling_params
)
user_prompts = [
    "Are there any immediate weather-related risks in my area that could disrupt network connectivity or system availability?",
]
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    session_id = agent.create_session("web-session")
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Are there any immediate weather-related risks in my area that could disrupt network connectivity or system availability?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': "current weather risks in user's location"}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': "current weather risks in user's location",
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in user location',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'User', 'region': 'Aceh', 'country': 'Indonesia', 'lat': 4.8659, 'lon': 96.6198, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1748036798, 'localtime': '2025-05-24 04:46'}, 'current': {'last_updated_epoch': 1748036700, 'last_updated': '2025-05-24 04:45', 'temp_c': 16.9, 'temp_f': 62.4, 'is_day': 0, 'condition': {'text': 'Fog', 'icon': '//cdn.weatherapi.com/weather/64x64/night/248.png', 'code': 1135}, 'wind_mph': 4.0, 'wind_kph': 6.5, 'wind_degree': 187, 'wind_dir': 'S', 'pressure_mb': 1010.0, 'pressure_in': 29.81, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 97, 'cloud': 100, 'feelslike_c': 16.9, 'feelslike_f': 62.4, 'windchill_c': 16.9, 'windchill_f': 62.4, 'heatindex_c': 16.9, 'heatindex_f': 62.4, 'dewpoint_c': 16.4, 'dewpoint_f': 61.6, 'vis_km': 0.0, 'vis_miles': 0.0, 'uv': 0.0, 'gust_mph': 8.5, 'gust_kph': 13.6}}",
│   │   │   'score': 0.7547392,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Track Severe Storms: The StormNet® Severe Weather Browser',
│   │   │   'url': 'https://www.iweathernet.com/weathernet-severe-weather-browser',
│   │   │   'content': 'Stay informed with up-to-the-minute data on current temperatures, wind speeds, and wind directions, while also monitoring highs and lows for the day. Most importantly, prioritize your safety by staying aware of current severe weather hazards, including tornado and flood warnings and watches, prominently displayed on the map.',
│   │   │   'score': 0.3217162,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': '7-Day Forecast 41.9N 88.23W - National Weather Service',
│   │   │   'url': 'https://forecast.weather.gov/MapClick.php?lat=41.8959291&lon=-88.21690269999999&site=all',
│   │   │   'content': 'A 20 percent chance of showers and thunderstorms after 4pm. Sunny, with a high near 88. Windy, with a south wind 10 to 20 mph increasing to 20 to 30 mph in the afternoon. Winds could gust as high as 40 mph.',
│   │   │   'score': 0.19319338,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Central Illinois - National Weather Service',
│   │   │   'url': 'https://www.weather.gov/ilx/',
│   │   │   'content': 'Last Map Update: Thu, May 22, 2025 at 5:22:25 pm CDTNOAA National Weather Service Central Illinois',
│   │   │   'score': 0.12350323,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Active Alerts - National Weather Service',
│   │   │   'url': 'https://www.weather.gov/alerts/as.htmlThere',
│   │   │   'content': 'US Dept of Commerce National Oceanic and Atmospheric Administration National Weather Service 1325 East West Highway Silver Spring, MD 20910 Comments? Questions? Please Contact Us.',
│   │   │   'score': 0.06037975,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Based on the latest weather updates, there are no immediate weather-related risks in your area that could disrupt network connectivity or system availability. The current weather in your location is foggy with a temperature of 62.4°F (16.9°C). There are no severe weather warnings or alerts issued by the National Weather Service. However, wind speeds could gust up to 40 mph later in the day. It's always a good idea to stay updated with local weather forecasts for any changes.

========== Query processing completed ========== 



### Output Analysis

In this example, since the agent is unaware of the users location, it hallucinates one and generates an incorrect search query. This misidentification leads to inaccurate information about potential weather-related risks.

This is where Prompt Chaining comes in. Prompt chaining allows the agent to:
1. Maintain context across multiple queries
2. Chain multiple tools together
3. Use previous interactions to inform current decisions

Let’s see how prompt chaining can improve the accuracy of the response.

## 3. Prompt chaining with websearch tool and client tool

In this section, we demonstrate a more sophisticated use case that combines the use of two tools: location detection and web search.

1. **Automatic Location Detection**: Use the `get_location` client tool to automatically determine the user's current location.
2. **Contextual Search**: Leverage the detected location to formulate the correct websearch query.

For example, when a user asks "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?", the agent will:
- First detect the user's current location using `get_location`.
- Then use that location to search for nearby weather-related risks.
- Finally, present a comprehensive response.

This demonstrates how the builtin websearch tool and custom client tools can work together to provide intelligent, context-aware responses without requiring explicit location input from the user.

In [6]:
agent = Agent(
    client, 
    model=model_id,
    instructions="""You are a helpful assistant. 
    When a user asks about their location, you MUST use the get_location tool. When you are asked to search the latest news, you MUST use the websearch tool.
    """ ,
    tools=[get_location, "builtin::websearch"],
    sampling_params=sampling_params
)
user_prompts = [
    "Where am I?",
    "Are there any immediate weather-related risks in my area that could disrupt network connectivity or system availability?"
]
session_id = agent.create_session("prompt-chaining-session")  # for prompt chaining, queries must share the same session_id.
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )

    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Where am I?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: get_location, Arguments: {'query': 'current location'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


'Your current location is: Boardman, Oregon, US'


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
You are currently located in Boardman, Oregon, US.

========== Query processing completed ========== 


Processing user query: Are there any immediate weather-related risks in my area that could disrupt network connectivity or system availability?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'Boardman, Oregon immediate weather risks'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'Boardman, Oregon immediate weather risks',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Boardman, Oregon',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Boardman', 'region': 'Oregon', 'country': 'United States of America', 'lat': 45.84, 'lon': -119.6994, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1748036885, 'localtime': '2025-05-23 14:48'}, 'current': {'last_updated_epoch': 1748036700, 'last_updated': '2025-05-23 14:45', 'temp_c': 22.8, 'temp_f': 73.0, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 6.7, 'wind_kph': 10.8, 'wind_degree': 234, 'wind_dir': 'SW', 'pressure_mb': 1016.0, 'pressure_in': 30.01, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 31, 'cloud': 0, 'feelslike_c': 24.0, 'feelslike_f': 75.1, 'windchill_c': 24.5, 'windchill_f': 76.1, 'heatindex_c': 24.7, 'heatindex_f': 76.5, 'dewpoint_c': 8.3, 'dewpoint_f': 46.9, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 6.5, 'gust_mph': 7.7, 'gust_kph': 12.4}}",
│   │   │   'score': 0.5431413,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Weather Alerts for Boardman, OR - The Weather Channel',
│   │   │   'url': 'https://weather.com/weather/alerts/localalerts/l/Boardman+OR?canonicalCityId=658113c17ad28bda472236e0fc229db7',
│   │   │   'content': 'Quick access to active weather alerts throughout Boardman, OR from The Weather Channel and Weather.com',
│   │   │   'score': 0.52583206,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Boardman, OR Weather Conditions | Weather Underground',
│   │   │   'url': 'https://www.wunderground.com/weather/us/or/boardman',
│   │   │   'content': 'Boardman Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the Boardman area.',
│   │   │   'score': 0.5012457,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Boardman OR Severe Weather Alerts & Warnings - LocalConditions.com',
│   │   │   'url': 'https://www.localconditions.com/us/boardman/oregon/weather/alerts/',
│   │   │   'content': 'We may provide a wider range of issued severe weather warnings for surrounding cities and counties, so some alerts may not directly impact this location. This is done to help with preparing for severe weather that may overlap or be heading towards Boardman. Use the Radar Maps link provided to understand the trajectory of a storm.',
│   │   │   'score': 0.48630774,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Boardman, OR Weather Advisories - Warnings & Watches | AccuWeather',
│   │   │   'url': 'https://www.accuweather.com/en/us/boardman/97818/weather-warnings/2187321',
│   │   │   'content': 'See a list of all of the Official Weather Advisories, Warnings, and Severe Weather Alerts for Boardman, OR.',
│   │   │   'score': 0.46614134,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
According to the latest weather reports, there are no immediate weather-related risks in Boardman, Oregon that could disrupt network connectivity or system availability. The current weather is sunny with a temperature of 73.0°F (22.8°C). There are no active weather alerts or warnings for the area at this time. However, it's always a good idea to stay updated with local weather forecasts for any changes.

========== Query processing completed ========== 



### ReAct Agent with websearch tool and client tool

This section demonstrates the ReAct (Reasoning and Acting) framework in action.

Here is a walkthrough of how the ReAct agent will tackle this same "weather near me" problem:

When asked "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?", the agent will:

1. **Reason** that it needs to get location information first.
2. **Act** by calling the `get_location` client tool.
3. **Observe** the location result.
4. **Reason** that it now needs to search for weather in that location.
5. **Act** by calling the `websearch` tool with observed location.
6. **Observe** and processes the search results into a final answer. 

Unlike prompt chaining which follows fixed steps, ReAct dynamically breaks down tasks and adapts its approach based on the results of each step. This makes it more flexible and capable of handling complex, real-world queries effectively.

In [7]:
agent = ReActAgent(
            client=client,
            model=model_id,
            tools=[get_location, "builtin::websearch"],
            response_format={
                "type": "json_schema",
                "json_schema": ReActOutput.model_json_schema(),
            },
            sampling_params=sampling_params,
        )
user_prompts = [
    "Are there any immediate weather-related risks in my area that could disrupt network connectivity or system availability?"
]
session_id = agent.create_session("web-session")
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Are there any immediate weather-related risks in my area that could disrupt network connectivity or system availability?

---------- 📍 Step 1: InferenceStep ----------
🤖 Model Response:
{
    "thought": "I need to get the user's current location and then check for any weather-related risks in that area. I will use the `get_location` tool to find the user's location and then the `web_search` tool to check for any weather-related risks.",
    "action": {
        "tool_name": "get_location",
        "tool_params": [{"name": "query", "value": "user's current location"}]
    },
    "answer": null
}


---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


'Your current location is: Boardman, Oregon, US'


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
{
    "thought": "Now that I have the user's location, I will use the `web_search` tool to check for any weather-related risks in Boardman, Oregon.",
    "action": {
        "tool_name": "web_search",
        "tool_params": [{"name": "query", "value": "weather-related risks in Boardman, Oregon"}]
    },
    "answer": null
}


---------- 📍 Step 4: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'weather-related risks in Boardman, Oregon',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Boardman, Oregon',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Boardman', 'region': 'Oregon', 'country': 'United States of America', 'lat': 45.84, 'lon': -119.6994, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1748037084, 'localtime': '2025-05-23 14:51'}, 'current': {'last_updated_epoch': 1748036700, 'last_updated': '2025-05-23 14:45', 'temp_c': 22.8, 'temp_f': 73.0, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 6.7, 'wind_kph': 10.8, 'wind_degree': 234, 'wind_dir': 'SW', 'pressure_mb': 1016.0, 'pressure_in': 30.01, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 31, 'cloud': 0, 'feelslike_c': 24.0, 'feelslike_f': 75.1, 'windchill_c': 24.5, 'windchill_f': 76.1, 'heatindex_c': 24.7, 'heatindex_f': 76.5, 'dewpoint_c': 8.3, 'dewpoint_f': 46.9, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 6.5, 'gust_mph': 7.7, 'gust_kph': 12.4}}",
│   │   │   'score': 0.7786702,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Top Oregon Climate Change Risks: Precipitation, Heat, Fire',
│   │   │   'url': 'https://climatecheck.com/oregon',
│   │   │   'content': 'Drought Risk in Oregon with Climate Change Drought risk is based on water stress , which is a projection of how much of the water supply will be taken up by human demand. In the figure, the blue bars represent the available water every 10 years from 2020-2060, and the orange bars represent demand.',
│   │   │   'score': 0.60245985,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': "OCCRI's Sixth Climate Assessment Outlines Climate Change Effects on Oregon",
│   │   │   'url': 'https://energyinfo.oregon.gov/blog/2023/1/11/occris-sixth-climate-assessment-outlines-climate-change-effects-on-oregon',
│   │   │   'content': "On January 4, 2023 the Oregon Climate Change Research Institute released the Sixth Oregon Climate Assessment, a biennial evaluation of recent scientific findings relating to the impacts and risks of climate change in Oregon.The Sixth Climate Assessment describes how climate change is affecting Oregon's environment and natural systems, economy, and communities, and presents projections of",
│   │   │   'score': 0.46920466,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': "Department of Land Conservation and Development : Oregon's Natural ...",
│   │   │   'url': 'https://www.oregon.gov/lcd/NH/Pages/Natural-Hazards.aspx',
│   │   │   'content': "The Oregon Natural Hazards Mitigation Plan (2015) identifies and addresses five additional hazards: droughts, dust storms, volcanoes, windstorms, and winter storms. Local governments may choose to address other hazards as well. Learn more about Oregon's natural hazards and how local communities can use land use planning to mitigate risks.",
│   │   │   'score': 0.4169159,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Climate change will keep hitting Oregonians hard, but the exact impact ...',
│   │   │   'url': 'https://www.opb.org/article/2024/09/02/climate-change-effects-oregon-research/',
│   │   │   'content': 'Oregon researchers say climate change will continue to have wide-ranging effects on communities across the state, depending on their region. May 20, 2025 contribute now',
│   │   │   'score': 0.2666399,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 5: InferenceStep ----------
🤖 Model Response:
{
    "thought": "Based on the search results, there are no immediate weather-related risks in Boardman, Oregon that could disrupt network connectivity or system availability. However, there are long-term climate change risks such as droughts and heatwaves.",
    "action": null,
    "answer": "There are currently no immediate weather-related risks in Boardman, Oregon that could disrupt network connectivity or system availability. However, long-term climate change risks such as droughts and heatwaves are present."
}

========== Query processing completed ========== 



## Key Takeaways
- This notebook demonstrated how to build more capable agents using Prompt Chaining and the ReAct framework.
- It showed how agents can maintain context across multiple steps and perform structured, multi-step reasoning.
- It highlights how ReAct enables dynamic tool selection and adaptive decision-making based on intermediate results.
- These techniques enhance agent autonomy and make them more suitable for complex operational tasks.

For further extensions, continue exploring in the next notebook: [RAG Agents](Level4_RAG_agent.ipynb).

#### Any Feedback?

If you have any feedback on this or any other notebook in this demo series we'd love to hear it! Please go to https://www.feedback.redhat.com/jfe/form/SV_8pQsoy0U9Ccqsvk and help us improve our demos. 